# Basic Policy Gradients (REINFORCE) example in PyTorch

Adaptation of lab 6 by Víctor Campos to PyTorch

## Installing dependences



In [0]:
!apt-get update
!apt-get install -y python-numpy python-dev cmake zlib1g-dev libjpeg-dev xvfb ffmpeg xorg-dev python-opengl libboost-all-dev libsdl2-dev swig
!pip install gym pyvirtualdisplay


Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1710/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1710/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  Release
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [83.2 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [1,364 B]
Get:13 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [119 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-updates/multiverse amd6

## Torch + torchvision installation (Py3.6)

In [0]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision

    100% |████████████████████████████████| 483.0MB 48.5MB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5745e000 @  0x7f058fe8f2a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
  Using cached https://files.pythonhosted.org/packages/ca/0d/f00b2885711e08bd71242ebe7b96561e6f6d01fdb4b9dcf4d37e2e13c5e1/torchvision-0.2.1-py2.py3-none-any.whl
    100% |████████████████████████████████| 2.0MB 12.6MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
import torch # Check if everything is OK

## Setting up the environment

In [44]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation
from IPython.display import HTML

# Start virtual display
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1024, 768))
display.start()
import os
os.environ["DISPLAY"] = ":" + str(display.display) + "." + str(display.screen)
from collections import namedtuple
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from torch.autograd import Variable

env = gym.make('CartPole-v0')

print("Action space (discrete): {}".format( env.action_space.n))
print("Observation space (discrete): {}".format( env.observation_space))

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Action space (discrete): 2
Observation space (discrete): Box(4,)


/usr/local/lib/python3.6/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


## Create the model

We will keep the same network use on lab 6

In [0]:
class PolicyNN(nn.Module):
  def __init__(self,obs_dim=4, num_actions=2):
    super(PolicyNN, self).__init__()
    self.fc1 = nn.Sequential(
      nn.Linear(obs_dim, 16),
      nn.Tanh())
    self.fc2 = nn.Sequential(
      nn.Linear(16, 16),
      nn.Tanh())
    self.fc3 =  nn.Linear(16, num_actions)

  def forward(self, x):
    x = self.fc1(x)
    x = self.fc2(x)
    logprobs = self.fc3(x)
    return F.softmax(logprobs)

In [0]:
class Agent:
  def __init__(self, config):
    self.env = gym.make(config.env)
    self.pi = PolicyNN()
    self.gamma = config.gamma  # discount factor for future rewards
    self.batch_size = config.batch_size  # in 4trajectories (episodes)
    self.optimizer = torch.optim.Adam(self.pi.parameters(), lr=config.learning_rate) 
    #self.optimizer = tf.train.AdamOptimizer(learning_rate=config.learning_rate)
    
    # Set random seed for reproducibility
    self.env.seed(config.seed)
    #tf.set_random_seed(config.seed)
    torch.manual_seed(config.seed)

    self.pi.train() # DONT FORGET TO PUT MODEL TO TRAIN ON INIT!!!!!!!
  
  
  def save(self, filename):
    """ Save policy weights to a ***.h5 file. """
    self.pi.save_state_dict(filename)
    #self.pi.save_weights(filename)
    
  def load(self, filename):
    """ Load policy weights from a ***.h5 file. """
    #self.pi.load_weights(filename)
    self.pi.load_state_dict(torch.load(filename))
    

  def select_action(self, state):
    probs = self.pi(Variable(state))       
    action=np.random.choice(self.env.action_space.n, p=probs.numpy)
    prob = probs[:, action[0,0]].view(1, -1)
    log_prob = prob.log()
    entropy = - (probs*probs.log()).sum()
    return action[0], log_prob, entropy
  
  def play(self, render):
    """ 
    Run the trained policy on the env. 
    Returns the rewards and the rendered frames for visualization.
    """
    ob = self.env.reset()
    done = False
    rews, frames = [], []
    while not done:
      if render:
        frames.append(self.env.render(mode='rgb_array'))
      # We need to convert ob to a tensor, and add a batch dimension...
      ob = tf.constant(np.array(ob)[None])
      # ... and remove it again once we are done with TF ops
      ac = self._sample_action(self.pi(ob)[0])
      ob, rew, done, info = self.env.step(ac)
      rews.append(rew)
    return rews, frames
  
  def _discount_rewards(self, rews):
    """ 
    Compute discounted rewards recursively. 
    Recall that with discount, r_t = sum_{k=t}^{T} gamma^{k-t} * r_k
    """
    discounted_rews = []
    last_r = 0.
    for r in reversed(rews):
      last_r = last_r * self.gamma + r
      discounted_rews.append(last_r)
    discounted_rews.reverse()
    return discounted_rews
  
  def _compute_gradients_tf(self, obs, actions, advantages):
    """
    Implement the policy gradient in TF. In order to do so with automatic
    differentiation, we need to build a surrogate loss whose gradient is 
    the policy gradient.
    This is achieved by taking the logprob of the selected action and 
    scaling it by the discounted reward.
    """
    advantages = np.array(advantages)  # [batch_size]
    actions = np.array(actions)  # [batch_size]
    with tf.GradientTape() as tape:
      logprobs = self.pi(np.array(obs)) # [batch_size, num_actions]
      # Compute PG surrogate and  flip sign for gradient ascent
      policy_grad_surrogate = - tf.nn.sparse_softmax_cross_entropy_with_logits(
          logits=logprobs, labels=actions)
      # Scale PG by the advantages
      print(policy_grad_surrogate)
      loss = tf.reduce_mean(advantages * policy_grad_surrogate)
    return tape.gradient(loss, self.pi.variables)
  
  def _compute_gradients_torch(self, obs, actions, advantages):
    """
    Implement the policy gradient in Torch. In order to do so with automatic
    differentiation, we need to build a surrogate loss whose gradient is 
    the policy gradient.
    This is achieved by taking the logprob of the selected action and 
    scaling it by the discounted reward.
"""
    R = torch.zeros(1, 1)
    loss = 0
    for i in reversed(range(len(rewards))):
        R = self.gamma * R + rewards[i]
        loss = loss - (log_probs[i]*(Variable(R).expand_as(log_probs[i]))).sum() - (0.0001*entropies[i].cuda()).sum()
    loss = loss / len(rewards)

    self.optimizer.zero_grad()
    loss.backward()
    utils.clip_grad_norm(self.model.parameters(), 40)
    self.optimizer.step()
    return R
                         
  def train(self, num_episodes):
    """
    Training loop:
      1. Collect experience by running the policy on the env
      2. Estimate the policy gradient with samples
      3. Update the policy and go back to (1)
    """
    rew_hist=[]
    for i_episode in range(num_episodes):
        state = torch.Tensor([self.env.reset()])
        entropies = []
        log_probs = []
        rewards = []
        done=False
        while not done:
            action, log_prob, entropy = self.select_action(state)
            action = action

            next_state, reward, done, _ = self.env.step(action.numpy()[0])

            entropies.append(entropy)
            log_probs.append(log_prob)
            rewards.append(reward)
            state = torch.Tensor([next_state])

            if done:
                break
    
        ep_rew=self._compute_gradients_torch(rewards, log_probs, entropies)
        if i_episode%args.ckpt_freq == 0:
            torch.save(agent.model.state_dict(), os.path.join(dir, 'reinforce-'+str(i_episode)+'.pkl'))

        print("Episode: {}, reward: {}".format(i_episode, np.sum(rewards)))
        rew_hist.append(ep_rew)

In [56]:
class CartPoleConfig:
  env = "CartPole-v0"
  seed = 0  # for reproducibility
  gamma = 0.99  # discount factor for future rewards
  batch_size = 4  # in trajectories (episodes)
  learning_rate = 0.001

agent = Agent(CartPoleConfig)

# Training on 3000 episodes takes around 3:30min on CPU
rew_history = agent.train(num_episodes=100)

/usr/local/lib/python3.6/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


TypeError: ignored